In [ ]:
# # imports
# !pip install stable-baselines3 pyglet imageio-ffmpeg gymnasium[atari,accept-rom-license] ale-py > /dev/null 2>&1
!pip install gymnasium==1.0.0
!pip install ale_py
!pip install stable-baselines3[extra]
# pip install sb3-contrib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.3 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import ale_py

In [4]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecTransposeImage
from stable_baselines3.common.atari_wrappers import MaxAndSkipEnv
import torch
import numpy as np
import wandb
from wandb.integration.sb3 import WandbCallback

from gymnasium.wrappers import MaxAndSkipObservation, ResizeObservation, GrayscaleObservation, FrameStackObservation, ReshapeObservation

In [5]:
from stable_baselines3.common.monitor import Monitor

In [6]:
import matplotlib.pyplot as plt

In [7]:
import os

In [8]:
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import CheckpointCallback

In [9]:
gym.register_envs(ale_py)

In [ ]:
# from sb3_contrib import QRDQN

In [10]:
from datetime import datetime
from stable_baselines3 import A2C
from stable_baselines3.ppo.policies import MlpPolicy
from wandb.integration.sb3 import WandbCallback

In [11]:
# ENV_NAME = 'ALE/MarioBros-v5'

# configuration file
config = {
    "policy_type": "CnnPolicy",
    "total_timesteps": 1000000,
    "env_name": "ALE/MarioBros-v5",
    "model_name": "A2C_MarioBros",
    "export_path": "./exports/",
    "videos_path": "./videos/",
}



In [12]:
# Wandb setup
run = wandb.init(
    project="Project_1",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    save_code=True,  # optional
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: amrit0081 (education_ai). Use `wandb login --relogin` to force relogin


In [13]:

class ScaledFloatFrame(gym.ObservationWrapper):
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0

class FireResetEnv(gym.Wrapper):
    def __init__(self, env=None):
        super().__init__(env)
        # Check that 'FIRE' is a valid action in the environment
        assert 'FIRE' in env.unwrapped.get_action_meanings(), "Environment does not support 'FIRE' action"
        assert len(env.unwrapped.get_action_meanings()) >= 3, "Action space too small for expected actions"

    def step(self, action):
        return self.env.step(action)

    def reset(self, **kwargs):
        # Reset the environment
        obs, info = self.env.reset(**kwargs)

        # Perform the FIRE action
        obs, _, terminated, truncated, _ = self.env.step(1)
        if terminated or truncated:  # If game ends after FIRE, reset again
            obs, info = self.env.reset(**kwargs)

        return obs, info


def make_env(env_name, render_mode=None):
    def _init():
        env = gym.make(env_name, render_mode=render_mode)
        print("Standard Env.        : {}".format(env.observation_space.shape))
        env = FireResetEnv(env)
        print("FireResetEnv          : {}".format(env.observation_space.shape))
        env = ResizeObservation(env, (84, 84))
        print("ResizeObservation    : {}".format(env.observation_space.shape))
        env = GrayscaleObservation(env, keep_dim=True)
        print("GrayscaleObservation : {}".format(env.observation_space.shape))
        env = ScaledFloatFrame(env)
        print("ScaledFloatFrame     : {}".format(env.observation_space.shape))
        env = Monitor(env, allow_early_resets=False) # from stable baselines
        print("Monitor               : {}".format(env.observation_space.shape))

        return env
    return _init



env = DummyVecEnv([make_env(config["env_name"])])
# stack 4 frames
env = VecFrameStack(env, n_stack=4)
print("Post VecFrameStack Shape: {}".format(env.observation_space.shape))

# convert back to PyTorch format (channel-first)
env = VecTransposeImage(env)
print("Final Observation Space: {}".format(env.observation_space.shape))

Standard Env.        : (210, 160, 3)
FireResetEnv          : (210, 160, 3)
ResizeObservation    : (84, 84, 3)
GrayscaleObservation : (84, 84, 1)
ScaledFloatFrame     : (84, 84, 1)
Monitor               : (84, 84, 1)
Post VecFrameStack Shape: (84, 84, 4)
Final Observation Space: (4, 84, 84)


In [14]:
# define
model = A2C(config["policy_type"], env, verbose=0, tensorboard_log=f"runs/{run.id}", device="cuda")

# train
t0 = datetime.now()
model.learn(total_timesteps=config["total_timesteps"], callback=WandbCallback(verbose=2))
t1 = datetime.now()
print('>>> Training time (hh:mm:ss.ms): {}'.format(t1-t0))

# save and export model
model.save(config["export_path"] + config["model_name"])


KeyboardInterrupt: 

In [ ]:
# !zip -r folder_name.zip exports
# from google.colab import files
# files.download('folder_name.zip')

  adding: exports/ (stored 0%)
  adding: exports/A2C_MarioBros.zip (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

# env DummyVecEnv([make_env(config["env_name"])])
env = make_env(config["env_name"])

# evaluate the agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print("Reward (mean +- std): {} +- {}".format(mean_reward, std_reward))

ValueError: The environment is of type <class 'function'>, not a Gymnasium environment. In this case, we expect OpenAI Gym to be installed and the environment to be an OpenAI Gym environment.

In [ ]:
# # Create output directory for saving models
# output_dir = "./dqn_mariobros/"
# os.makedirs(output_dir, exist_ok=True)

# # Define a checkpoint callback to save models during training
# checkpoint_callback = CheckpointCallback(save_freq=10000, save_path=output_dir, name_prefix="dqn_mariobros")

# # Initialize the DQN model
# model = DQN(
#     "CnnPolicy",            # Convolutional policy for image input
#     env,                    # Custom preprocessed environment
#     learning_rate=1e-4,     # Learning rate
#     buffer_size=100000,     # Replay buffer size
#     learning_starts=10000,  # Steps before training starts
#     batch_size=32,          # Batch size for training
#     gamma=0.99,             # Discount factor
#     target_update_interval=500,  # Target network update frequency
#     train_freq=4,           # Training frequency (in steps)
#     verbose=1,              # Verbosity level
#     tensorboard_log="./dqn_tensorboard/"  # TensorBoard log directory
# )

# # Train the DQN agent
# model.learn(total_timesteps=500000, callback=checkpoint_callback)

# # Save the final model
# model.save(os.path.join(output_dir, "dqn_mariobros_final"))

# # Test the trained model
# obs = env.reset()
# for _ in range(1000):
#     action, _ = model.predict(obs, deterministic=True)
#     obs, reward, done, info = env.step(action)
#     env.render()  # Render the environment
#     if done:
#         obs = env.reset()

# env.close()

In [ ]:
print(aaaa)

Testing

In [ ]:
env = DummyVecEnv([make_env(config["env_name"], render_mode="rgb_array")])
# stack 4 frames
env = VecFrameStack(env, n_stack=4)
print("Post VecFrameStack Shape: {}".format(env.observation_space.shape))

# convert back to PyTorch format (channel-first)
env = VecTransposeImage(env)
print("Final Observation Space: {}".format(env.observation_space.shape))

print("Render mode after wrapping:", env.render_mode)

Standard Env.        : (210, 160, 3)
FireResetEnv          : (210, 160, 3)
ResizeObservation    : (84, 84, 3)
GrayscaleObservation : (84, 84, 1)
ScaledFloatFrame     : (84, 84, 1)
Monitor               : (84, 84, 1)
Post VecFrameStack Shape: (84, 84, 4)
Final Observation Space: (4, 84, 84)
Render mode after wrapping: rgb_array


In [ ]:
# load model
model = A2C.load("A2C_MarioBros")


In [ ]:
import imageio
from PIL import Image
import PIL.ImageDraw as ImageDraw

rewards_glb = []
num_episodes = 1

for i in range(num_episodes):
    frames = []
    rewards_episode = []
    done = False
    obs = env.reset()

    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        # done = terminated or truncated
        rewards_episode.append(reward)

        frames.append(env.render())

    rewards_glb.append(sum(rewards_episode))
    # e.g. fps=50 == duration=20 (1000 * 1/50)
    imageio.mimwrite("model_name" +'_'+ str(i) +'.gif', frames, duration=20)

print("Rewards:", rewards_glb)



Rewards: [array([1600.], dtype=float32)]


End TESTING

In [ ]:
# # Play a few steps and inspect the observations
# obs = env.reset()  # Reset the environment
# print("Initial Observation Shape:", obs.shape)
# print("Non-Zero Values in Initial Observation:", np.count_nonzero(obs))

# # Take 10 random steps
# for step in range(10):
#     action = [env.action_space.sample()]
#     obs, reward, done, info = env.step(action)
#     print(f"Step {step+1}:")
#     print("  Observation Shape:", obs.shape)
#     print("  Non-Zero Values:", np.count_nonzero(obs))

# # Check if everything is correctly handled
# obs[0]  # Display one observation after the steps

In [ ]:
env.observation_space.shape

In [ ]:
env.action_space

In [ ]:
# # play 1 step
# obs = env.reset()
# print(obs.shape)

# # select random action
# action = env.action_space.sample()
# # play the action
# obs, reward, q, info = env.step([3])
# print(obs.shape)
# obs = obs.squeeze()
# print(obs.shape)
# print(obs.min(), obs.max())
# # count total
# print(np.count_nonzero(obs))
# obs

In [ ]:
# class FireResetEnv(gym.Wrapper):
#     def __init__(self, env=None):
#         super(FireResetEnv, self).__init__(env)
#         assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
#         assert len(env.unwrapped.get_action_meanings()) >= 3

#     def step(self, action):
#         return self.env.step(action)

#     def reset(self):
#         self.env.reset()
#         obs, _, terminated, truncated, _ = self.env.step(1)
#         done = terminated or truncated
#         if done:
#             self.env.reset()
#         return obs

In [ ]:
# import gymnasium as gym
# import ale_py
# from gymnasium.wrappers import MaxAndSkipObservation, ResizeObservation, GrayscaleObservation, FrameStackObservation, ReshapeObservation

# import numpy as np

# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torchsummary import summary

# import collections

# import wandb
# import datetime

In [ ]:
ENV_NAME = 'ALE/MarioBros-v5'

In [ ]:
class ImageToPyTorch(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], old_shape[0], old_shape[1]), dtype=np.float32)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class ScaledFloatFrame(gym.ObservationWrapper):
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0


def make_env(env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    print("Standard Env.        : {}".format(env.observation_space.shape))
    env = MaxAndSkipObservation(env, skip=4)
    print("MaxAndSkipObservation: {}".format(env.observation_space.shape))
    # env = FireResetEnv(env)
    # print("FireResetEnv         : {}".format(env.observation_space.shape))
    env = ResizeObservation(env, (84, 84))
    print("ResizeObservation    : {}".format(env.observation_space.shape))
    env = GrayscaleObservation(env, keep_dim=True)
    print("GrayscaleObservation : {}".format(env.observation_space.shape))
    env = ImageToPyTorch(env)
    print("ImageToPyTorch       : {}".format(env.observation_space.shape))
    env = ReshapeObservation(env, (84, 84))
    print("ReshapeObservation   : {}".format(env.observation_space.shape))
    env = FrameStackObservation(env, stack_size=4)
    print("FrameStackObservation: {}".format(env.observation_space.shape))
    env = ScaledFloatFrame(env)
    print("ScaledFloatFrame     : {}".format(env.observation_space.shape))

    return env

env=make_env(ENV_NAME)

In [ ]:
# play one step of the game
obs = env.reset()
# select random action
action = env.action_space.sample()
# play the action
obs, reward, t,t, info = env.step(3)
obs

In [ ]:
# check different values in obs
print("Min: ", obs.min())
print("Max: ", obs.max())

In [ ]:
obs.shape

In [ ]:
# Count the number of values that are not zero
non_zero_count = np.count_nonzero(obs)

non_zero_count

In [ ]:
# select random action
action = env.action_space.sample()
# play the action
obs, reward, t,t, info = env.step(3)
obs

In [ ]:
# check different values in obs
print("Min: ", obs.min())
print("Max: ", obs.max())

In [ ]:
# Count the number of values that are not zero
non_zero_count = np.count_nonzero(obs)

non_zero_count